In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
data

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data["PassengerId"] = le.fit_transform(data["PassengerId"])
data["PassengerId"]

In [ ]:
# data["PassengerId"]=le.inverse_transform(data['PassengerId'])[:8693]
# data["PassengerId"]

In [ ]:
# data["PassengerId"]=data.PassengerId.astype("int")

In [ ]:
data.Transported.unique()

In [ ]:
data.head(2).T

In [ ]:
data.isnull().sum()

In [ ]:
data.Destination=data.Destination.fillna('None')
data.CryoSleep=data.CryoSleep.fillna(True)
data.VIP=data.VIP.fillna(True)
data.Name=data.Name.fillna('anonymous')
data.Age=data.Age.fillna(20)
data.RoomService=data.RoomService.fillna(20)
data.FoodCourt=data.FoodCourt.fillna(0)
data.ShoppingMall=data.ShoppingMall.fillna(1000)
data.Spa=data.Spa.fillna(1000)
data.VRDeck=data.VRDeck.fillna(20)
data.HomePlanet=data.HomePlanet.fillna('None')

In [ ]:
data['Destination']=data['Destination'].replace('TRAPPIST-1e','TRAPPIST').replace('PSO J318.5-22','PSO').replace('55 Cancri e','Canceri')
data['CryoSleep']=data['CryoSleep'].replace(True,1).replace(False,0)
data['VIP']=data['VIP'].replace(True,1).replace(False,0)
data['Transported']=data['Transported'].replace(False,0).replace(True,1)

In [ ]:
# data.PassengerId=data.PassengerId.astype("str")

In [ ]:
data.head(10)

In [ ]:
data.Cabin.unique

In [ ]:
data = data.astype({"Age":'int', "RoomService":'int',"FoodCourt":'int',"ShoppingMall":'int',"VRDeck":'int',"Spa":'int'})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
sns.set(rc = {'figure.figsize':(15,8)})
sns.countplot(x='HomePlanet',hue='Destination',data=data,palette='flare')

In [ ]:
sns.countplot(data.Transported)

In [ ]:
data.iloc[:, :-1].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='magma')\
                     .bar(subset=["max"], color='#BB0000')\
                     .bar(subset=["mean",], color='green')

In [ ]:
x=data.drop(columns=['Cabin','Transported','Name'])    #'PassengerId'
y=data.Transported

In [ ]:
x.CryoSleep.unique()

In [ ]:
x.isnull().sum()

In [ ]:
x

In [ ]:
data.HomePlanet.value_counts()

In [ ]:
data.HomePlanet.unique()

In [ ]:
# nom_cols=[0,2]
# ord_cols=[1,3,4,5,6,7,8,9]
nom_cols=[1,3]
num_cols=[4,5,6,7,8,9]
# ord_cols=[2,4,5,6,7,8,9,10]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn import set_config
from sklearn.experimental import enable_iterative_imputer
from catboost import CatBoostClassifier
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.preprocessing import Binarizer
#https://dataaspirant.com/catboost-algorithm/
from sklearn.pipeline import make_pipeline
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.35)
# trans=make_column_transformer((OneHotEncoder(),nom_cols),(CatBoostClassifier(verbose=True),cbc),
#                               (IterativeImputer(),null_cols),remainder='passthrough')
# set_config(display='diagram')
# trans
cat_features=list(range(0,x.shape[1]))
clf=CatBoostClassifier(iterations=350)
pipe=clf.fit(x_train, y_train,cat_features=cat_features,eval_set=(x_test, y_test),verbose=False)
pipe

In [ ]:
pred_clf=clf.predict(x_test)
pred_clf

In [ ]:
accuracy_score(pred_clf,y_test)*100

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# gbc=GradientBoostingClassifier()
# pipe_g=gbc.fit(x,y)

In [ ]:
trans=make_column_transformer((OneHotEncoder(sparse=False),nom_cols),(PowerTransformer(),num_cols),remainder='passthrough')   #(Binarizer(threshold=15),ord_cols)
set_config(display='diagram')
trans

In [ ]:
pipe_s=make_pipeline(trans,pipe)
pipe_s

In [ ]:
pipe_s.fit(x_train,y_train)

In [ ]:
pred_s=pipe_s.predict(x_test)
pred_s

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(pred_s,y_test)*100

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# gbc=GradientBoostingClassifier()
# pipe_g=make_pipeline(trans,gbc)
# pipe_g.fit(x_train,y_train)
# pred_g=pipe_g.predict(x_test)
# accuracy_score(pred_g,y_test)*100


In [ ]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
from imblearn.pipeline import make_pipeline
over=RandomOverSampler()
o_x,o_y=over.fit_resample(x,y)
s=SMOTE()
n_pipe=make_pipeline(trans,s,pipe)
n_pipe

In [ ]:
n_pipe.fit(x_train,y_train)

In [ ]:
n_pred=n_pipe.predict(x_test)
n_pred

In [ ]:
accuracy_score(n_pred,y_test)*100

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
pipe_g=make_pipeline(trans,s,gbc)
pipe_g.fit(x_train,y_train)
pred_g=pipe_g.predict(x_test)
accuracy_score(pred_g,y_test)*100


In [ ]:
data1=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
data1

In [ ]:
# data1["PassengerId"] = le.fit_transform(data1["PassengerId"])
# data1["PassengerId"]

In [ ]:
data1.isnull().sum()

In [ ]:
data1.Destination=data.Destination.fillna('None')
data1.CryoSleep=data.CryoSleep.fillna(True)
data1.VIP=data.VIP.fillna(True)
data1.Name=data.Name.fillna('anonymous')
data1.Age=data.Age.fillna(20)
data1.RoomService=data.RoomService.fillna(20)
data1.FoodCourt=data.FoodCourt.fillna(0)
data1.ShoppingMall=data.ShoppingMall.fillna(1000)
data1.Spa=data.Spa.fillna(1000)
data1.VRDeck=data.VRDeck.fillna(20)
data1.HomePlanet=data.HomePlanet.fillna('None')

In [ ]:
data1['Destination']=data1['Destination'].replace('TRAPPIST-1e','TRAPPIST').replace('PSO J318.5-22','PSO').replace('55 Cancri e','Canceri')
data1['CryoSleep']=data1['CryoSleep'].replace(True,1).replace(False,0)
data1['VIP']=data1['VIP'].replace(True,1).replace(False,0)
data1 = data1.astype({"Age":'int', "RoomService":'int',"FoodCourt":'int',"ShoppingMall":'int',"VRDeck":'int',"Spa":'int'})

In [ ]:
# data1["PassengerId"]=le.inverse_transform(data['PassengerId'])[:8693]
# data1["PassengerId"]

In [ ]:
x1=data1.drop(columns=['Name'],axis=1)    #'Cabin'
x1.head(10)

In [ ]:
x1["PassengerId"] = le.fit_transform(x1["PassengerId"])
x1["PassengerId"]

In [ ]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# data["PassengerId"] = le.fit_transform(data["PassengerId"])
# data1["PassengerId"]= le.fit_transform(data1["PassengerId"])

In [ ]:
# import xgboost as xgb
# my_model = xgb.XGBClassifier()
# my_model.fit(x_train, y_train)
   
# # Predicting the Test set results
# y_pred = my_model.predict(x_test)

In [ ]:
pred123=pipe_g.predict(x1)
pred123

In [ ]:
# data1["PassengerId"]=le.inverse_transform(data['PassengerId'])[:8693]
# data1["PassengerId"]
# data["PassengerId"]=le.inverse_transform(data['PassengerId'])[:8693]
# data["PassengerId"]

In [ ]:
submit=pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

In [ ]:
import pandas as pd                              
submit=pd.DataFrame({'PassengerId':data1['PassengerId'],'Transported':pred123.astype("bool")})              #submit['Transported']=pred123
submit.to_csv('submission.csv', index=False)

In [ ]:
submit.Transported.value_counts()

In [ ]:
data1.PassengerId.unique()

In [ ]:
submit.head(15)

In [ ]:
y_test.head(8)